In [ ]:
!pip install pyspark sparksql-magic "u8darts[pmdarima]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 814.5/814.5 kB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 50.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.8/279.8 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.2/169.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1VDWxdHrYNBnLs4_7lb7xeiPi-RsJmOkP' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1VDWxdHrYNBnLs4_7lb7xeiPi-RsJmOkP" -O dataset.csv && rm -rf /tmp/cookies.txt

--2023-11-23 12:29:39--  https://docs.google.com/uc?export=download&confirm=t&id=1VDWxdHrYNBnLs4_7lb7xeiPi-RsJmOkP
Resolving docs.google.com (docs.google.com)... 142.251.31.101, 142.251.31.113, 142.251.31.100, ...
Connecting to docs.google.com (docs.google.com)|142.251.31.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/648eijgqohtrguu01ls9aufc2hpscldo/1700742525000/07665009615024983083/*/1VDWxdHrYNBnLs4_7lb7xeiPi-RsJmOkP?e=download&uuid=42a10583-aa63-4d11-8acc-fa2d2da39d71 [following]
--2023-11-23 12:29:39--  https://doc-14-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/648eijgqohtrguu01ls9aufc2hpscldo/1700742525000/07665009615024983083/*/1VDWxdHrYNBnLs4_7lb7xeiPi-RsJmOkP?e=download&uuid=42a10583-aa63-4d11-8acc-fa2d2da39d71
Resolving doc-14-7o-docs.googleusercontent.com (doc-14-7o-docs.googleusercontent.com)... 108.177.119.132,

In [ ]:
%load_ext sparksql_magic

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring, lit, to_date, trim
from pyspark.sql import functions as f
import pandas as pd

spark = SparkSession.builder.appName("Hackaton").getOrCreate()

## National Data

In [ ]:
df = spark.read.csv("dataset.csv",  header=True, inferSchema=True, sep=";")

df_leitos = spark.read.csv("Leitos_2023.csv", header=True, inferSchema=True)\
  .select("LEITOS_EXISTENTES", "UTI_TOTAL_EXIST", "UTI_TOTAL_SUS", "COMP", "UF", "MUNICIPIO", "CNES")

df_leitos_2022 = spark.read.csv("Leitos_2022.csv", header=True, inferSchema=True)\
  .withColumnRenamed("LEITOS EXISTENTE", "LEITOS_EXISTENTES")\
  .withColumnRenamed("UTI TOTAL - EXIST", "UTI_TOTAL_EXIST")\
  .withColumnRenamed("UTI TOTAL - SUS", "UTI_TOTAL_SUS")\
  .select("LEITOS_EXISTENTES", "UTI_TOTAL_EXIST", "UTI_TOTAL_SUS", "COMP", "UF", "MUNICIPIO", "CNES")

df_leitos_2021 = spark.read.csv("Leitos_2021.csv", header=True, inferSchema=True)\
  .withColumnRenamed("LEITOS EXISTENTE", "LEITOS_EXISTENTES")\
  .withColumnRenamed("UTI TOTAL - EXIST", "UTI_TOTAL_EXIST")\
  .withColumnRenamed("UTI TOTAL - SUS", "UTI_TOTAL_SUS")\
  .select("LEITOS_EXISTENTES", "UTI_TOTAL_EXIST", "UTI_TOTAL_SUS", "COMP", "UF", "MUNICIPIO", "CNES")

df_leitos_2020 = spark.read.csv("Leitos_2020.csv", header=True, inferSchema=True)\
  .withColumnRenamed("LEITOS EXISTENTE", "LEITOS_EXISTENTES")\
  .withColumnRenamed("UTI TOTAL - EXIST", "UTI_TOTAL_EXIST")\
  .withColumnRenamed("UTI TOTAL - SUS", "UTI_TOTAL_SUS")\
  .select("LEITOS_EXISTENTES", "UTI_TOTAL_EXIST", "UTI_TOTAL_SUS", "COMP", "UF", "MUNICIPIO", "CNES")

df_leitos_2019 = spark.read.csv("Leitos_2019.csv", header=True, inferSchema=True)\
  .withColumnRenamed("LEITOS EXISTENTE", "LEITOS_EXISTENTES")\
  .withColumnRenamed("UTI TOTAL - EXIST", "UTI_TOTAL_EXIST")\
  .withColumnRenamed("UTI TOTAL - SUS", "UTI_TOTAL_SUS")\
  .select("LEITOS_EXISTENTES", "UTI_TOTAL_EXIST", "UTI_TOTAL_SUS", "COMP", "UF", "MUNICIPIO", "CNES")

all_leitos = df_leitos.unionByName(df_leitos_2022)\
  .unionByName(df_leitos_2021)\
  .unionByName(df_leitos_2020)\
  .unionByName(df_leitos_2019)\
  .withColumn("YEAR", substring(col("COMP"), 1,4))\
  .withColumn("date", to_date(col("COMP"), 'yyyyMM'))

df_municipios = spark.read.csv("municipios.csv", header=True, inferSchema=True)

df_dados_mensais = spark.read.csv("mensal.csv", header=True, inferSchema=True)

new_columns = [col(column).alias(column.strip()) for column in df_dados_mensais.columns]
df_dados_mensais = df_dados_mensais.select(*new_columns).drop("INDEX_IGNORE")


In [ ]:
all_leitos.createOrReplaceTempView("leitos_data")
df.createOrReplaceTempView("hackaton_data")
df_municipios.createOrReplaceTempView("municipios")
df_dados_mensais.createOrReplaceTempView("mensal")

In [ ]:
all_leitos.show()

+-----------------+---------------+-------------+------+---+--------------------+----+----+----------+
|LEITOS_EXISTENTES|UTI_TOTAL_EXIST|UTI_TOTAL_SUS|  COMP| UF|           MUNICIPIO|CNES|YEAR|      date|
+-----------------+---------------+-------------+------+---+--------------------+----+----+----------+
|               64|              0|            0|202301| PE|CABO DE SANTO AGO...|  27|2023|2023-01-01|
|               19|              0|            0|202301| PE|CABO DE SANTO AGO...|  35|2023|2023-01-01|
|               18|              0|            0|202301| PE|CABO DE SANTO AGO...|  94|2023|2023-01-01|
|               59|              5|            0|202301| PE|CABO DE SANTO AGO...| 183|2023|2023-01-01|
|               47|             10|            0|202301| PE|CABO DE SANTO AGO...| 221|2023|2023-01-01|
|              427|             20|           20|202301| PE|              RECIFE| 396|2023|2023-01-01|
|              410|             59|           59|202301| PE|             

In [ ]:
distinct_municipios = spark.sql("""
  select distinct ID_MUNICIP
  from hackaton_data
""")

In [ ]:
main_df = spark.sql("""
with step1 as (
  select
    SG_UF_NOT UF,ID_MUNICIP,
    count(distinct CO_UNI_NOT) UNIDADES
  from hackaton_data
    where ID_MUNICIP is not null and CO_UNI_NOT is not null
  group by 1,2
  order by 1
  ),

  leitos_aggregated as (
  select
    UF, MUNICIPIO,
    sum(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
    sum(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
    sum(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
  from leitos_data
  where YEAR = '2023'
  group by 1,2
  ),

 muni_changed as (
    select
      TRIM(UPPER(uf_code)) UF,
      CASE
        WHEN TRIM(UPPER(no_accents)) = 'ITAPAJE' THEN 'ITAPAGE'
        WHEN TRIM(UPPER(no_accents)) = 'SAO JOAO DEL-REI' THEN 'SAO JOAO DEL REI'
        WHEN TRIM(UPPER(no_accents)) = 'ELDORADO DO CARAJAS' THEN 'ELDORADO DOS CARAJAS'
        WHEN TRIM(UPPER(no_accents)) = 'BELEM DO SAO FRANCISCO' THEN 'BELEM DE SAO FRANCISCO'
        WHEN TRIM(UPPER(no_accents)) = 'PARATY' THEN 'PARATI'
        WHEN TRIM(UPPER(no_accents)) = 'AÇU' THEN 'ACU'
        WHEN TRIM(UPPER(no_accents)) = "SANT'ANA DO LIVRAMENTO" THEN 'SANTANA DO LIVRAMENTO'
        WHEN TRIM(UPPER(no_accents)) = 'MOGI MIRIM' THEN 'MOJI MIRIM'
        ELSE TRIM(UPPER(no_accents))
      END as MUNICIPIO,
      pop_21 POPULATION
    from municipios m
  ),

  muni_used as (
    select * from muni_changed m
    left semi join step1 h
    on TRIM(UPPER(m.UF)) = TRIM(UPPER(h.UF)) and TRIM(UPPER(h.ID_MUNICIP)) = TRIM(UPPER(m.MUNICIPIO))
  )

  select s.*,
  mu.POPULATION,
  coalesce(l.LEITOS_EXISTENTES, 0) LEITOS_EXISTENTES,
  coalesce(l.UTI_TOTAL_SUS, 0) UTI_TOTAL_SUS,
  coalesce(l.UTI_TOTAL_EXIST,  0) UTI_TOTAL_EXIST
  from step1 s

  left join muni_used mu
  on TRIM(UPPER(mu.UF)) = TRIM(UPPER(s.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(mu.MUNICIPIO))

  left join leitos_aggregated l
  on TRIM(UPPER(s.UF)) = TRIM(UPPER(l.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(l.MUNICIPIO))

  order by UF
""")

In [ ]:
main_df.toPandas().to_csv("data_2023_v1.csv", index=False)

In [ ]:
year_diffs = spark.sql("""
with step1 as (
  select
    SG_UF_NOT UF,ID_MUNICIP,
    count(distinct CO_UNI_NOT) UNIDADES
  from hackaton_data
    where ID_MUNICIP is not null and CO_UNI_NOT is not null
  group by 1,2
  order by 1
  ),

  leitos_aggregated as (
  select
    UF, MUNICIPIO,
    sum(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
    sum(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
    sum(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
  from leitos_data
  where YEAR = '2023'
  group by 1,2
  ),

 muni_changed as (
    select
      TRIM(UPPER(uf_code)) UF,
      CASE
        WHEN TRIM(UPPER(no_accents)) = 'ITAPAJE' THEN 'ITAPAGE'
        WHEN TRIM(UPPER(no_accents)) = 'SAO JOAO DEL-REI' THEN 'SAO JOAO DEL REI'
        WHEN TRIM(UPPER(no_accents)) = 'ELDORADO DO CARAJAS' THEN 'ELDORADO DOS CARAJAS'
        WHEN TRIM(UPPER(no_accents)) = 'BELEM DO SAO FRANCISCO' THEN 'BELEM DE SAO FRANCISCO'
        WHEN TRIM(UPPER(no_accents)) = 'PARATY' THEN 'PARATI'
        WHEN TRIM(UPPER(no_accents)) = 'AÇU' THEN 'ACU'
        WHEN TRIM(UPPER(no_accents)) = "SANT'ANA DO LIVRAMENTO" THEN 'SANTANA DO LIVRAMENTO'
        WHEN TRIM(UPPER(no_accents)) = 'MOGI MIRIM' THEN 'MOJI MIRIM'
        ELSE TRIM(UPPER(no_accents))
      END as MUNICIPIO,
      pop_21 POPULATION
    from municipios m
  ),

  muni_used as (
    select * from muni_changed m
    left semi join step1 h
    on TRIM(UPPER(m.UF)) = TRIM(UPPER(h.UF)) and TRIM(UPPER(h.ID_MUNICIP)) = TRIM(UPPER(m.MUNICIPIO))
  )

  select s.*,
  mu.POPULATION,
  coalesce(l.LEITOS_EXISTENTES, 0) LEITOS_EXISTENTES,
  coalesce(l.UTI_TOTAL_SUS, 0) UTI_TOTAL_SUS,
  coalesce(l.UTI_TOTAL_EXIST,  0) UTI_TOTAL_EXIST
  from step1 s

  left join muni_used mu
  on TRIM(UPPER(mu.UF)) = TRIM(UPPER(s.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(mu.MUNICIPIO))

  left join leitos_aggregated l
  on TRIM(UPPER(s.UF)) = TRIM(UPPER(l.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(l.MUNICIPIO))

  order by UF
""")

### SQL Query

In [ ]:
%%sparksql %%sql
create or replace temporary view first_table as
with step1 as (
  select
    SG_UF_NOT UF,ID_MUNICIP,
    count(distinct CO_UNI_NOT) UNIDADES
  from hackaton_data
    where ID_MUNICIP is not null and SUPORT_VEN is not null
    and CLASSI_FIN in (4,5)
  group by 1,2
  order by 1
  ),

  leitos_aggregated as (
  select
    UF, MUNICIPIO,
    sum(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
    sum(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
    sum(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
  from leitos_data
  group by 1,2
  ),

  leitos_2022_agg as (
  select
    UF, MUNICIPIO,
    sum(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
    sum(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
    sum(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
  from leitos_2022_data
  group by 1,2
  ),

 muni_changed as (
    select
      TRIM(UPPER(uf_code)) UF,
      CASE
        WHEN TRIM(UPPER(no_accents)) = 'ITAPAJE' THEN 'ITAPAGE'
        WHEN TRIM(UPPER(no_accents)) = 'SAO JOAO DEL-REI' THEN 'SAO JOAO DEL REI'
        WHEN TRIM(UPPER(no_accents)) = 'ELDORADO DO CARAJAS' THEN 'ELDORADO DOS CARAJAS'
        WHEN TRIM(UPPER(no_accents)) = 'BELEM DO SAO FRANCISCO' THEN 'BELEM DE SAO FRANCISCO'
        WHEN TRIM(UPPER(no_accents)) = 'PARATY' THEN 'PARATI'
        WHEN TRIM(UPPER(no_accents)) = 'AÇU' THEN 'ACU'
        WHEN TRIM(UPPER(no_accents)) = "SANT'ANA DO LIVRAMENTO" THEN 'SANTANA DO LIVRAMENTO'
        WHEN TRIM(UPPER(no_accents)) = 'MOGI MIRIM' THEN 'MOJI MIRIM'
        ELSE TRIM(UPPER(no_accents))
      END as MUNICIPIO,
      pop_21 POPULATION
    from municipios m
  ),

  muni_used as (
    select * from muni_changed m
    left semi join step1 h
    on TRIM(UPPER(m.UF)) = TRIM(UPPER(h.UF)) and TRIM(UPPER(h.ID_MUNICIP)) = TRIM(UPPER(m.MUNICIPIO))
  )

  select s.*,
  mu.POPULATION,
  coalesce(l.LEITOS_EXISTENTES, o.LEITOS_EXISTENTES, 0) LEITOS_EXISTENTES,
  coalesce(l.UTI_TOTAL_SUS, o.UTI_TOTAL_SUS, 0) UTI_TOTAL_SUS,
  coalesce(l.UTI_TOTAL_EXIST, o.UTI_TOTAL_EXIST, 0) UTI_TOTAL_EXIST
  from step1 s

  left join muni_used mu
  on TRIM(UPPER(mu.UF)) = TRIM(UPPER(s.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(mu.MUNICIPIO))

  left join leitos_aggregated l
  on TRIM(UPPER(s.UF)) = TRIM(UPPER(l.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(l.MUNICIPIO))

  left join leitos_2022_agg o
  on TRIM(UPPER(s.UF)) = TRIM(UPPER(o.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(o.MUNICIPIO))

  order by UF

capture dataframe to local variable `%%sql`


In [ ]:
%%sparksql %%sql
create or replace temporary view leitos_by_years as
WITH leitos_aggregated AS (
  SELECT
    UF, MUNICIPIO, date,
    SUM(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
    SUM(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
    SUM(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
  FROM leitos_data
  GROUP BY 1,2,3
),

muni_changed AS (
  SELECT
    TRIM(UPPER(uf_code)) UF,
    CASE
      WHEN TRIM(UPPER(no_accents)) = 'ITAPAJE' THEN 'ITAPAGE'
      WHEN TRIM(UPPER(no_accents)) = 'SAO JOAO DEL-REI' THEN 'SAO JOAO DEL REI'
      WHEN TRIM(UPPER(no_accents)) = 'ELDORADO DO CARAJAS' THEN 'ELDORADO DOS CARAJAS'
      WHEN TRIM(UPPER(no_accents)) = 'BELEM DO SAO FRANCISCO' THEN 'BELEM DE SAO FRANCISCO'
      WHEN TRIM(UPPER(no_accents)) = 'PARATY' THEN 'PARATI'
      WHEN TRIM(UPPER(no_accents)) = 'AÇU' THEN 'ACU'
      WHEN TRIM(UPPER(no_accents)) = "SANT'ANA DO LIVRAMENTO" THEN 'SANTANA DO LIVRAMENTO'
      WHEN TRIM(UPPER(no_accents)) = 'MOGI MIRIM' THEN 'MOJI MIRIM'
      ELSE TRIM(UPPER(no_accents))
    END AS MUNICIPIO,
    pop_21 POPULATION
  FROM municipios m
),

muni_used AS (
  SELECT * FROM muni_changed m
  LEFT SEMI JOIN leitos_aggregated h
  ON TRIM(UPPER(m.UF)) = TRIM(UPPER(h.UF)) AND TRIM(UPPER(h.MUNICIPIO)) = TRIM(UPPER(m.MUNICIPIO))
),

leitos_with_diff AS (
  SELECT
    s.date, s.UF, s.MUNICIPIO,
    mu.POPULATION,
    COALESCE(s.LEITOS_EXISTENTES, 0) LEITOS_EXISTENTES,
    COALESCE(s.UTI_TOTAL_SUS, 0) UTI_TOTAL_SUS,
    COALESCE(s.UTI_TOTAL_EXIST,  0) UTI_TOTAL_EXIST
    --COALESCE(s.LEITOS_EXISTENTES, 0) - LAG(COALESCE(s.LEITOS_EXISTENTES, 0), 1) OVER (PARTITION BY s.UF, s.MUNICIPIO ORDER BY s.YEAR) AS LEITOS_DIFF
  FROM leitos_aggregated s
  LEFT JOIN muni_used mu
  ON TRIM(UPPER(mu.UF)) = TRIM(UPPER(s.UF)) AND TRIM(UPPER(s.MUNICIPIO)) = TRIM(UPPER(mu.MUNICIPIO))
)

SELECT date, UF, MUNICIPIO,
  POPULATION,LEITOS_EXISTENTES,	UTI_TOTAL_SUS, UTI_TOTAL_EXIST
FROM leitos_with_diff
WHERE UF = 'DF'
ORDER BY MUNICIPIO, date


capture dataframe to local variable `%%sql`


In [ ]:
%%sparksql %%sql
select * from leitos_by_years order by date

capture dataframe to local variable `%%sql`
only showing top 20 row(s)


date,UF,MUNICIPIO,POPULATION,LEITOS_EXISTENTES,UTI_TOTAL_SUS,UTI_TOTAL_EXIST
2019-01-01,DF,BRASILIA,3094325,8310,281,1124
2019-02-01,DF,BRASILIA,3094325,8272,281,1106
2019-03-01,DF,BRASILIA,3094325,8384,281,1151
2019-04-01,DF,BRASILIA,3094325,8756,281,1209
2019-05-01,DF,BRASILIA,3094325,8801,281,1219
2019-06-01,DF,BRASILIA,3094325,8692,281,1248
2019-07-01,DF,BRASILIA,3094325,8759,281,1260
2019-08-01,DF,BRASILIA,3094325,8630,281,1260
2019-09-01,DF,BRASILIA,3094325,8600,281,1248
2019-10-01,DF,BRASILIA,3094325,8579,281,1305


In [ ]:
import plotly.express as px

In [ ]:
%%sparksql %%sql
--create or replace temporary view main_data as
with step1 as (
select
  SG_UF_NOT UF,ID_MUNICIP, CO_UNI_NOT CNES,
  count(distinct CO_UNI_NOT) UNIDADES,
  count(*) PACIENTES,
  sum(
    case when EVOLUCAO not in (1,9) then 1
    else 0 end
  ) OBITOS,
  sum(
    case when (SUPORT_VEN not in (1,2) and EVOLUCAO not in (1,9)) then 1
    else 0 end
  ) OSS,
  OBITOS - OSS OCS
from hackaton_data
  where ID_MUNICIP is not null and SUPORT_VEN is not null
  and CLASSI_FIN in (4,5)
group by 1,2,3
order by 1
),

leitos_aggregated as (
select
  UF, MUNICIPIO, CNES,
  sum(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
  sum(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
  sum(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
from leitos_data
where YEAR = 2023
group by 1,2,3
),

brasilia as (
select s.*, l.LEITOS_EXISTENTES, l.UTI_TOTAL_SUS, l.UTI_TOTAL_EXIST
from step1 s
left join leitos_aggregated l
on TRIM(UPPER(s.UF)) = TRIM(UPPER(l.UF)) and TRIM(UPPER(s.ID_MUNICIP)) = TRIM(UPPER(l.MUNICIPIO)) and s.CNES = l.CNES
order by ID_MUNICIP
)

select *
from brasilia
where UF = 'DF'

capture dataframe to local variable `%%sql`
only showing top 20 row(s)


UF,ID_MUNICIP,CNES,UNIDADES,PACIENTES,OBITOS,OSS,OCS,LEITOS_EXISTENTES,UTI_TOTAL_SUS,UTI_TOTAL_EXIST
DF,BRASILIA,7116756,1,4,2,0,2,null,null,null
DF,BRASILIA,6876617,1,570,17,3,14,1980,380,380
DF,BRASILIA,10472,1,24,7,0,7,4146,200,200
DF,BRASILIA,10480,1,269,16,1,15,5570,140,200
DF,BRASILIA,6730914,1,2,2,0,2,320,200,200
DF,BRASILIA,10456,1,53,15,0,15,7216,680,796
DF,BRASILIA,3030121,1,1,0,0,0,2170,0,950
DF,BRASILIA,935514,1,65,12,1,11,null,null,null
DF,BRASILIA,7465157,1,6,6,0,6,null,null,null
DF,BRASILIA,2672197,1,41,11,2,9,1636,200,270


In [ ]:
%%sparksql %%sql
create or replace temporary view leitos_por_cnes as
WITH leitos_aggregated AS (
  SELECT
    UF, MUNICIPIO, date ,CNES,
    SUM(LEITOS_EXISTENTES) LEITOS_EXISTENTES,
    SUM(UTI_TOTAL_SUS) UTI_TOTAL_SUS,
    SUM(UTI_TOTAL_EXIST) UTI_TOTAL_EXIST
  FROM leitos_data
  GROUP BY 1,2,3,4
),

muni_changed AS (
  SELECT
    TRIM(UPPER(uf_code)) UF,
    CASE
      WHEN TRIM(UPPER(no_accents)) = 'ITAPAJE' THEN 'ITAPAGE'
      WHEN TRIM(UPPER(no_accents)) = 'SAO JOAO DEL-REI' THEN 'SAO JOAO DEL REI'
      WHEN TRIM(UPPER(no_accents)) = 'ELDORADO DO CARAJAS' THEN 'ELDORADO DOS CARAJAS'
      WHEN TRIM(UPPER(no_accents)) = 'BELEM DO SAO FRANCISCO' THEN 'BELEM DE SAO FRANCISCO'
      WHEN TRIM(UPPER(no_accents)) = 'PARATY' THEN 'PARATI'
      WHEN TRIM(UPPER(no_accents)) = 'AÇU' THEN 'ACU'
      WHEN TRIM(UPPER(no_accents)) = "SANT'ANA DO LIVRAMENTO" THEN 'SANTANA DO LIVRAMENTO'
      WHEN TRIM(UPPER(no_accents)) = 'MOGI MIRIM' THEN 'MOJI MIRIM'
      ELSE TRIM(UPPER(no_accents))
    END AS MUNICIPIO,
    pop_21 POPULATION
  FROM municipios m
),

muni_used AS (
  SELECT * FROM muni_changed m
  LEFT SEMI JOIN leitos_aggregated h
  ON TRIM(UPPER(m.UF)) = TRIM(UPPER(h.UF)) AND TRIM(UPPER(h.MUNICIPIO)) = TRIM(UPPER(m.MUNICIPIO))
),

leitos_with_diff AS (
  SELECT
    s.date, s.CNES, s.UF, s.MUNICIPIO,
    mu.POPULATION,
    COALESCE(s.LEITOS_EXISTENTES, 0) LEITOS_EXISTENTES,
    COALESCE(s.UTI_TOTAL_SUS, 0) UTI_TOTAL_SUS,
    COALESCE(s.UTI_TOTAL_EXIST,  0) UTI_TOTAL_EXIST
    --COALESCE(s.LEITOS_EXISTENTES, 0) - LAG(COALESCE(s.LEITOS_EXISTENTES, 0), 1) OVER (PARTITION BY s.UF, s.MUNICIPIO ORDER BY s.YEAR) AS LEITOS_DIFF
  FROM leitos_aggregated s
  LEFT JOIN muni_used mu
  ON TRIM(UPPER(mu.UF)) = TRIM(UPPER(s.UF)) AND TRIM(UPPER(s.MUNICIPIO)) = TRIM(UPPER(mu.MUNICIPIO))
)

SELECT date,CNES,LEITOS_EXISTENTES,	UTI_TOTAL_SUS, UTI_TOTAL_EXIST
FROM leitos_with_diff
WHERE UF = 'DF'
ORDER BY CNES

capture dataframe to local variable `%%sql`


In [ ]:
%%sparksql %%sql
create or replace temporary view ocupacao_data as
with step1 as (
select *, CAST(CONCAT(SUBSTRING(DT_INTERNA, 7, 4), '-', SUBSTRING(DT_INTERNA, 4, 2), '-', SUBSTRING(DT_INTERNA, 1, 2)) as DATE) DATA_ATT,DATE(DATE_TRUNC('month',DATA_ATT)) as MONTH, CO_UNI_NOT CNES
from hackaton_data
  where SG_UF_NOT = "DF" and ID_MUNICIP is not null and DT_INTERNA is not null
),

step2 as (
  select *, CAST(CONCAT(SUBSTRING(DT_EVOLUCA, 7, 4), '-', SUBSTRING(DT_EVOLUCA, 4, 2), '-', SUBSTRING(DT_EVOLUCA, 1, 2)) as DATE) DATA_ATT,DATE(DATE_TRUNC('month',DATA_ATT)) as MONTH, CO_UNI_NOT CNES
  from hackaton_data
  where SG_UF_NOT = "DF" and ID_MUNICIP is not null and DT_EVOLUCA is not null
),

entradas_hosp as (
select MONTH, CNES,
COUNT(*) as ENTRADAS_HOSP
from step1
where HOSPITAL = 1
group by 1,2
order by 1 asc
),

saidas_hosp as (
  select MONTH, CNES,
  COUNT(*) as SAIDAS_HOSP
  from step2
  where HOSPITAL = 1
  group by 1,2
  order by 1 asc
),

uti_data1 as (
  select *, CAST(CONCAT(SUBSTRING(DT_ENTUTI, 7, 4), '-', SUBSTRING(DT_ENTUTI, 4, 2), '-', SUBSTRING(DT_ENTUTI, 1, 2)) as DATE) DATA_ATT,DATE(DATE_TRUNC('month',DATA_ATT)) as MONTH, CO_UNI_NOT CNES
  from hackaton_data
  where SG_UF_NOT = "DF" and ID_MUNICIP is not null and DT_ENTUTI is not null
),

uti_data2 as (
  select *, CAST(CONCAT(SUBSTRING(DT_SAIDUTI, 7, 4), '-', SUBSTRING(DT_SAIDUTI, 4, 2), '-', SUBSTRING(DT_SAIDUTI, 1, 2)) as DATE) DATA_ATT,DATE(DATE_TRUNC('month',DATA_ATT)) as MONTH, CO_UNI_NOT CNES
  from hackaton_data
  where SG_UF_NOT = "DF" and ID_MUNICIP is not null and DT_SAIDUTI is not null
),

entradas_uti as (
select MONTH, CNES,
COUNT(*) as ENTRADAS_UTI
from uti_data1
where UTI=1
group by 1,2
order by 1 asc
),

saidas_uti as (
  select MONTH, CNES,
  COUNT(*) as SAIDAS_UTI
  from uti_data2
  where UTI = 1
  group by 1,2
  order by 1 asc
)


select e.*,
  coalesce(s.SAIDAS_HOSP,0) SAIDAS_HOSP,
  coalesce(eu.ENTRADAS_UTI,0) ENTRADAS_UTI,
  coalesce(su.SAIDAS_UTI,0) SAIDAS_UTI
from entradas_hosp e
left join saidas_hosp s
on e.MONTH = s.MONTH and e.CNES  = s.CNES

left join entradas_uti eu
on e.MONTH = eu.MONTH and e.CNES  = eu.CNES

left join saidas_uti su
on e.MONTH = su.MONTH and e.CNES  = su.CNES

capture dataframe to local variable `%%sql`


In [ ]:
%%sparksql %%sql
select * from leitos_por_cnes
where date = '2019-01-01' and CNES = 1120

capture dataframe to local variable `%%sql`


date,CNES,LEITOS_EXISTENTES,UTI_TOTAL_SUS,UTI_TOTAL_EXIST


In [ ]:
%%sparksql %%sql
select date(DT), int(CNES) from mensal

capture dataframe to local variable `%%sql`
only showing top 20 row(s)


DT,CNES
2019-01-01,1120
2019-02-01,1120
2019-03-01,1120
2019-04-01,1120
2019-05-01,1120
2019-06-01,1120
2019-07-01,1120
2019-08-01,1120
2019-09-01,1120
2019-10-01,1120


In [ ]:
%%sparksql %%sql
create or replace temporary view final_df as
select m.*,
  coalesce(l.LEITOS_EXISTENTES,0.0) LEITOS_EXISTENTES,
  coalesce(l.UTI_TOTAL_SUS,0.0) UTI_TOTAL_SUS,
  coalesce(l.UTI_TOTAL_EXIST,0.0) UTI_TOTAL_EXIST
from mensal m
left join leitos_por_cnes l
on l.date = date(m.DT) and l.CNES = int(m.CNES)
order by date

capture dataframe to local variable `%%sql`


In [ ]:
dates = pd.date_range(start='2019-01-01', end='2024-12-01', freq='MS')
covariables = pd.DataFrame({'date': dates})

covariables['inverno'] = 0
covariables.loc[(covariables['date'].dt.month >= 7) & (covariables['date'].dt.month <= 9), 'inverno'] = 1

In [ ]:
covariables

In [ ]:
brasilia_data = spark.table("final_df")\
.withColumn("date", to_date(col("DT")))\
.drop("DT")\
.toPandas()

brasilia_data['date'] = pd.to_datetime(brasilia_data['date'])

In [ ]:
brasilia_data['LEITOS_EXISTENTES'] = brasilia_data['LEITOS_EXISTENTES'].astype('float64')
brasilia_data['UTI_TOTAL_SUS'] = brasilia_data['UTI_TOTAL_SUS'].astype('float64')
brasilia_data['UTI_TOTAL_EXIST'] = brasilia_data['UTI_TOTAL_EXIST'].astype('float64')

brasilia_data['CNES'] = brasilia_data['CNES'].astype('int').astype('str')

brasilia_data.set_index('date', inplace=True)

In [ ]:
brasilia_data.reset_index

In [ ]:
resultados_filtrados = brasilia_data[brasilia_data['CNES'] =='7978642'].sort_values('date')

In [ ]:
resultados_filtrados

,CNES,HOSPITAL,UTI,SURTO_SG,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,...,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OUT_MORBI,SUPORT_VEN,LEITOS_EXISTENTES,UTI_TOTAL_SUS,UTI_TOTAL_EXIST,inverno
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,0
2019-02-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,0
2019-03-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,0
2019-04-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,0
2019-05-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,0
2019-06-01,7978642,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,2.0,140.0,0.0,36.0,0
2019-07-01,7978642,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,140.0,0.0,36.0,1
2019-08-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,1
2019-09-01,7978642,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,140.0,0.0,36.0,1


In [ ]:
px.line(resultados_filtrados[['HOSPITAL', 'UTI', 'LEITOS_EXISTENTES']])

## Forecast

In [ ]:
cnes = brasilia_data.groupby('CNES').CNES.count().sort_values(ascending=False)
cnes = cnes.loc[lambda x : x >= 18] # Filter the series with at least 12 months long]
cnes

CNES
101826     52
471755     52
5310350    52
5253381    52
5200105    52
           ..
2393565    52
2389746    52
2383926    52
2383462    52
996068     52
Name: CNES, Length: 308, dtype: int64

### SARIMA (deu bom n)

In [ ]:
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models.forecasting.sf_auto_arima import StatsForecastAutoARIMA
from dateutil.relativedelta import relativedelta

scaler = Scaler()
mf = StatsForecastAutoARIMA(season_length=12, **{'seasonal': True})

In [ ]:
# TODO: ADD coluna inverno no mês para variaveis exogenas e também feriados (influencia ??)

In [ ]:
results_df = []
errors = []
horizon = 6 # forecast

for unidade in cnes.index:
  try:
    print(f'\nCNES: {unidade}')
    df_group = brasilia_data[brasilia_data.CNES == unidade]
    df_temp = df_group.copy()

    df_temp.drop(['CNES'],axis=1, inplace=True)

    # limitate the extension to 1 year
    mindate = df_temp.reset_index()['date'].min()
    df_temp = df_temp[df_temp.index >= mindate+relativedelta(months=-12)]
    df_temp = TimeSeries.from_dataframe(df_temp)

    cov_df = TimeSeries.from_dataframe(covariables).add_datetime_attribute(attribute='year').add_datetime_attribute(attribute='month')

    mf.fit(df_temp.strip(), future_covariates=cov_df)

    hist = mf.historical_forecasts(df_temp.strip(), future_covariates=cov_df, forecast_horizon=horizon, start=0.6, num_samples=500, verbose=False)

    forecast = mf.predict(horizon, future_covariates=cov_df, num_samples=500)


  except Exception as e:
    errors.append((unidade, e))



CNES: 10456

CNES: 2645157

CNES: 2649527

CNES: 2672197

CNES: 2673916

CNES: 6876617

CNES: 2814897

CNES: 2815966

CNES: 6730914

CNES: 3013162

CNES: 3018520

CNES: 3019616

CNES: 6362729

CNES: 3048551

CNES: 10464

CNES: 3276678

CNES: 6243495

CNES: 5831628

CNES: 5717515

CNES: 5085047

CNES: 2649497

CNES: 5027438

CNES: 9699848

CNES: 10502

CNES: 10510

CNES: 10529

CNES: 10537

CNES: 10545

CNES: 10618

CNES: 10480

CNES: 10499

CNES: 9040021

CNES: 7981457

CNES: 7978642

CNES: 10472

CNES: 9727353

CNES: 3019608

CNES: 3000974

CNES: 6921434

CNES: 7978243

CNES: 3005402

CNES: 3025020

CNES: 3030121

CNES: 9551948

CNES: 5796423

CNES: 7242441

CNES: 9307524

CNES: 7344538

CNES: 9748202

CNES: 3055450

CNES: 9928553

CNES: 9914234

CNES: 7477090

CNES: 7865791

CNES: 7907958

CNES: 49867

CNES: 10561

CNES: 7897634

CNES: 5927579

CNES: 7766777

CNES: 185256

CNES: 223441

CNES: 9052682

CNES: 7161158

CNES: 552216

CNES: 797146

CNES: 3035379

CNES: 252492

CNES: 9501

In [ ]:
fdf = pd.concat(results_df)

### Prophet

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from prophet.plot import plot_cross_validation_metric

In [ ]:
covariables = brasilia_data.copy()[[col for col in brasilia_data.columns if col != "HOSPITAL"]].sort_values('date')
covariables

,CNES,UTI,SURTO_SG,NOSOCOMIAL,AVE_SUINO,FEBRE,TOSSE,GARGANTA,DISPNEIA,DESC_RESP,...,PNEUMOPATI,IMUNODEPRE,RENAL,OBESIDADE,OUT_MORBI,SUPORT_VEN,LEITOS_EXISTENTES,UTI_TOTAL_SUS,UTI_TOTAL_EXIST,inverno
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,5882451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2019-01-01,3450198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2019-01-01,49867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2019-01-01,10464,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,357.0,16.0,16.0,0
2019-01-01,8003629,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-01,2396998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2023-04-01,5882451,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2023-04-01,6854729,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
results = []
errors = []

for unidade in cnes.index:
  print(f'\nCNES: {unidade}')
  df_group = brasilia_data[brasilia_data.CNES == unidade]
  df_temp = df_group.copy()

  print(df_temp.head(5))

  break


CNES: 101826
              CNES  HOSPITAL  UTI  SURTO_SG  NOSOCOMIAL  AVE_SUINO  FEBRE  \
date                                                                        
2019-03-01  101826       0.0  0.0       0.0         0.0        0.0    0.0   
2022-04-01  101826       0.0  0.0       0.0         0.0        0.0    0.0   
2021-04-01  101826       0.0  0.0       0.0         0.0        0.0    0.0   
2022-09-01  101826       0.0  0.0       0.0         0.0        0.0    0.0   
2022-08-01  101826       0.0  0.0       0.0         0.0        0.0    0.0   

            TOSSE  GARGANTA  DISPNEIA  ...  PNEUMOPATI  IMUNODEPRE  RENAL  \
date                                   ...                                  
2019-03-01    0.0       0.0       0.0  ...         0.0         0.0    0.0   
2022-04-01    0.0       0.0       0.0  ...         0.0         0.0    0.0   
2021-04-01    0.0       0.0       0.0  ...         0.0         0.0    0.0   
2022-09-01    0.0       0.0       0.0  ...         0.0       

## DF Data

In [ ]:
import os

In [ ]:
file_names = ["transferir.csv"] + [f"transferir ({i}).csv" for i in range(1, 12)]
dataframes = []
for i, f in enumerate(file_names):
  df = spark.read.csv(f, header=True, inferSchema=True).withColumn("Month", lit(i+1))
  dataframes.append(df)

merged_df = dataframes[0]
for df in dataframes[1:]:
    merged_df = merged_df.union(df)

pandas_df = merged_df.toPandas()

output_csv_path = "2020_DF_Data.csv"
pandas_df.to_csv(output_csv_path, index=False)

In [ ]:
df = spark.read.csv("2020_DF_Data.csv", header=True, inferSchema=True).filter((col('i_desc_uf_res') == 'DF') & (col('i_mes_compt') >= 3))

In [ ]:
all_years_df = spark.read.csv("DF_DATA_ALL.csv", header=True, inferSchema=True).filter((col('i_desc_uf_res') == 'DF'))

In [ ]:
corona_df = df.filter(df.i_desc_proc_realizado == 'TRATAMENTO DE INFECÇÃO PELO CORONAVIRUS – COVID 19')\
.union(all_years_df.filter(all_years_df.i_desc_proc_realizado == 'TRATAMENTO DE INFECÇÃO PELO CORONAVIRUS – COVID 19'))

In [ ]:
corona_df.createOrReplaceTempView("corona_data")

In [ ]:
corona_df.orderBy('i_ano_compt', ascending=False).show()

+-----------+---------------+-----------+--------------------+--------------------+-------------------+------------------+--------------------+----------------+------------+---------------------+----------------+--------------------+-----------+-----------------------+--------------------+-------------+------------+--------------+-----------+--------+----------------+---------+----------------+----------------+-----------+-------+-----+
|i_ano_compt|i_cid_principal|i_cirurgias|i_desc_car_int_atend|i_desc_cid_principal|i_desc_complex_proc|i_desc_espec_leito|        i_desc_grupo|i_desc_munic_res|i_desc_obito|i_desc_proc_realizado| i_desc_radf_res| i_desc_regiao_saude|i_desc_sexo|i_desc_sigla_estab_cnes|  i_desc_tipo_financ|i_desc_uf_res|i_estab_cnes|i_faixa_etaria|i_mes_compt|i_partos|i_proc_realizado|i_qtd_aih|i_qtd_diaria_pac|i_qtd_diaria_uti|i_valor_aih|     id|Month|
+-----------+---------------+-----------+--------------------+--------------------+-------------------+---------------

In [ ]:
%%sparksql %%sql
create or replace temporary view pacientes_by_ra as
select DATE(CONCAT(i_ano_compt, '-', i_mes_compt, '-01')) AS date, i_desc_radf_res RA,count(*) pacientes
from corona_data
group by 1,2
order by 1

capture dataframe to local variable `%%sql`


In [ ]:
%%sparksql %%sql
SELECT * FROM pacientes_by_ra

capture dataframe to local variable `%%sql`
only showing top 20 row(s)


date,RA,pacientes
2020-04-01,Guará,2
2020-04-01,Plano Piloto,5
2020-04-01,Lago Sul,2
2020-04-01,Samambaia,1
2020-04-01,Brazlândia,1
2020-04-01,Recanto das Emas,1
2020-04-01,Ceilândia,6
2020-04-01,São Sebastião,1
2020-04-01,Candangolândia,1
2020-04-01,Santa Maria,9


In [ ]:
%%sparksql %%sql
with corona_att as (
  select *, DATE(CONCAT(i_ano_compt, '-', i_mes_compt, '-01')) AS date
  from corona_data
)

SELECT
    t1.date,
    t1.RA,
    t1.pacientes,
    t1.pacientes / t2.total_pacientes AS proporcao
FROM
    pacientes_by_ra t1
JOIN
    (SELECT date, COUNT(*) total_pacientes
    FROM corona_att
    GROUP BY date) t2
ON t1.date = t2.date
ORDER BY proporcao desc

capture dataframe to local variable `%%sql`
only showing top 20 row(s)


date,RA,pacientes,proporcao
2022-09-01,Samambaia,1,1.0
2022-07-01,Ceilândia,5,0.5
2022-06-01,Ceilândia,10,0.4166666666666667
2022-08-01,Ceilândia,2,0.3333333333333333
2022-05-01,Ceilândia,58,0.26851851851851855
2020-04-01,Santa Maria,9,0.24324324324324326
2022-07-01,null,2,0.2
2021-11-01,Ceilândia,144,0.1909814323607427
2022-01-01,Ceilândia,61,0.17579250720461095
2022-08-01,Lago Norte,1,0.16666666666666666


In [ ]:
%%sparksql %%sql
create or replace temporary view proporcao_by_month as
WITH corona_att AS (
  SELECT *, DATE(CONCAT(i_ano_compt, '-', i_mes_compt, '-01')) AS date
  FROM corona_data
),
proporcoes AS (
  SELECT
      t1.date,
      t1.RA,
      t1.pacientes,
      t1.pacientes / t2.total_pacientes AS proporcao
  FROM
      pacientes_by_ra t1
  JOIN
      (SELECT date, COUNT(*) total_pacientes
      FROM corona_att
      GROUP BY date) t2
  ON t1.date = t2.date
)
SELECT
    RA,
    EXTRACT(MONTH FROM date) AS mes,
    AVG(proporcao) AS media_proporcao
FROM proporcoes
where RA is not null
GROUP BY RA, mes
ORDER BY media_proporcao DESC



capture dataframe to local variable `%%sql`


In [ ]:
spark.table('proporcao_by_month').toPandas().to_csv('proporcao_by_month.csv', index=False)